In [1]:
import json
import itertools
import time
import os
import re
import numpy as np
import tensorflow as tf

In [2]:
DATA_PATH = os.path.expanduser(".")

# Vocab

`vocab.json` is pre-built, it's a list of top 100k identifier names.

In [3]:
with open("{}/vocab.json".format(DATA_PATH), 'r') as j:
    words = json.load(j)
    words.append('<unk>')       # Unknown identifiers, low-freq identifiers outside vocabulary
    words.append('<flag>')      # End of script symbol, if script is flagged
    words.append('<no-flag>')   # End of script symbol, if script is not flagged
    vocab = dict(zip(words, range(len(words))))

In [4]:
vocab_len = len(words)
words[:24]

[u'length',
 u'prototype',
 u'push',
 u'document',
 u'window',
 u'options',
 u'data',
 u'call',
 u'type',
 u'arguments',
 u'value',
 u'name',
 u'replace',
 u'style',
 u'Math',
 u'css',
 u'width',
 u'height',
 u'indexOf',
 u'test',
 u'top',
 u'element',
 u'apply',
 u'event']

In [5]:
len(words) == len(vocab)

True

In [5]:
print("Size of Vocab : {}".format(vocab_len))

Size of Vocab : 200090


# Transform

Convert raw script tokens to array of integers (indices defined in `vocab` dict).

In [6]:
def digitize_script(label, tokens):
    if int(label) == 1:
        flag = '<flag>'
    else:
        flag = '<no-flag>'
    tokens = json.loads(tokens) + [flag] # append EOS symbol
    get_index = lambda w: vocab[w] if (w in vocab) else vocab['<unk>']
    return [get_index(t) for t in tokens]

In [8]:
def get_raw():
    with open("{}/tokens-labeled.jsonl".format(DATA_PATH), 'r') as f:
        for line in f.readlines():
            yield digitize_script(*line.strip().split("\t"))

In [9]:
def stringify(token_indices):
    return " ".join([words[i] for i in token_indices])

In [10]:
corpus = []
for tokens in get_raw():
    for t in tokens:
        corpus.append(t)

In [11]:
print("Corpus Size: {}".format(len(corpus)))

Corpus Size: 199187149


In [12]:
stringify(corpus[1000:1048])

u'{ [Keyword|this] . user_info . email = [Keyword|this] . email ; } [Keyword|else] { [Keyword|this] . user_info = { [String] : [Keyword|this] . email } ; } } [Keyword|this] . email_list_id = [Keyword|this] . getURLParameter ( [String] ) || [Keyword|this] . getURLParameter ( [String] ) ; [Keyword|this] .'

<hr>

# Begin Tensorflow Code.



In [13]:
def data_type():
  return tf.float32

def data_producer(raw_data, batch_size, num_steps, name=None):
    
  with tf.name_scope(name, "DataProducer", [raw_data, batch_size, num_steps]):
    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)

    data_len = tf.size(raw_data)
    batch_len = data_len // batch_size
    data = tf.reshape(raw_data[0 : batch_size * batch_len],
                      [batch_size, batch_len])

    epoch_size = (batch_len - 1) // num_steps
    assertion = tf.assert_positive(
        epoch_size,
        message="epoch_size == 0, decrease batch_size or num_steps")
    with tf.control_dependencies([assertion]):
      epoch_size = tf.identity(epoch_size, name="epoch_size")

    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = tf.slice(data, [0, i * num_steps], [batch_size, num_steps])
    y = tf.slice(data, [0, i * num_steps + 1], [batch_size, num_steps])
    return x, y

In [14]:
class Input(object):
  """The input data."""

  def __init__(self, config, data, name=None):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
    self.input_data, self.targets = data_producer(
        data, batch_size, num_steps, name=name)

In [15]:
class Seq2SeqModel:
    def __init__(self, config, is_training, input_, initial_state=None):

        self._input = input_

        batch_size = input_.batch_size
        num_steps = input_.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size

        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0, state_is_tuple=True)
        if is_training and config.keep_prob < 1:
            lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
                lstm_cell, output_keep_prob=config.keep_prob)
        cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers, state_is_tuple=True)

        with tf.device("/cpu:0"):
            embedding = tf.get_variable(
                "embedding", [vocab_size, size], dtype=data_type())
            inputs = tf.nn.embedding_lookup(embedding, input_.input_data)

        _initial_state = cell.zero_state(batch_size, data_type())

        if initial_state is not None:
            self._initial_state = tf.placeholder(dtype=data_type(), shape=tf.shape(_initial_state))
        else:
            self._initial_state = _initial_state

        # if is_training and config.keep_prob < 1:
        #  inputs = tf.nn.dropout(inputs, config.keep_prob)

        # ---- Notes ----
        # 't' is a tensor of shape [1, 2, 1, 3, 1, 1]
        # shape(squeeze(t, [0])) ==> [2, 1, 3, 1, 1]
        inputs = [tf.squeeze(input_step, [1])
                     for input_step in tf.split(1, num_steps, inputs)]
        outputs, state = tf.nn.rnn(cell, inputs, initial_state=self._initial_state, dtype=data_type())

        # shape: [num_steps, size]
        output = tf.reshape(tf.concat(1, outputs), [-1, size])

        softmax_w = tf.get_variable(
            "softmax_w", [size, vocab_size], dtype=data_type())
        softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=data_type())
        logits = tf.matmul(output, softmax_w) + softmax_b
        # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/seq2seq.py#L986
        targets = tf.reshape(input_.targets, [-1])
        loss = tf.nn.seq2seq.sequence_loss_by_example(
            [logits],
            [targets],
            [tf.ones([batch_size * num_steps], dtype=data_type())])
        self._cost = cost = tf.reduce_sum(loss) / batch_size
        self._final_state = state

        self._predict = probs = tf.nn.softmax(logits)
        predict_class = tf.cast(tf.argmax(probs, 1), tf.int32)
        
        errors = tf.cast(tf.equal(predict_class, targets), data_type())
        self._errors = tf.reduce_mean(errors)

        if not is_training:
          return

        self._lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                          config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self._lr)
        self._train_op = optimizer.apply_gradients(
            zip(grads, tvars),
            global_step=tf.contrib.framework.get_or_create_global_step())

        self._new_lr = tf.placeholder(
            tf.float32, shape=[], name="new_learning_rate")
        self._lr_update = tf.assign(self._lr, self._new_lr)

    def assign_lr(self, session, lr_value):
        session.run(self._lr_update, feed_dict={self._new_lr: lr_value})

    @property
    def input(self):
        return self._input

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state

    @property
    def lr(self):
        return self._lr
    
    @property
    def predict(self):
        return self._predict

    @property
    def train_op(self):
        return self._train_op

In [16]:
def run_epoch(session, model, eval_op=None, verbose=False, return_eval=False):
  """Runs the model on the given data."""
  start_time = time.time()
  costs = 0.0
  iters = 0
  state = session.run(model.initial_state)

  fetches = {
      "cost": model.cost,
      "final_state": model.final_state,
  }
  if eval_op is not None:
    fetches["eval_op"] = eval_op
  if return_eval:
    eval_results = []

  for step in range(model.input.epoch_size):
    feed_dict = {}
    for i, (c, h) in enumerate(model.initial_state):
      feed_dict[c] = state[i].c
      feed_dict[h] = state[i].h

    vals = session.run(fetches, feed_dict)
    cost = vals["cost"]
    state = vals["final_state"]

    costs += cost
    iters += model.input.num_steps
    
    if return_eval:
      eval_results.append(vals['eval_op'])

    if verbose and step % (model.input.epoch_size // 10) == 10:
      print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / model.input.epoch_size, np.exp(costs / iters),
             iters * model.input.batch_size / (time.time() - start_time)))

  if return_eval:
    return eval_results

  return np.exp(costs / iters)

In [17]:
class SmallConfig(object):
    """Small config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 10
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = len(words)

In [18]:
class MediumConfig(object):
    """Medium config."""
    init_scale = 0.05
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 650
    max_epoch = 6
    max_max_epoch = 39
    keep_prob = 0.7
    lr_decay = 0.8
    batch_size = 20
    vocab_size = len(words)

In [ ]:
config = MediumConfig()
# eval_config = get_config()
# eval_config.batch_size = 1
# eval_config.num_steps = 1
eval_config = MediumConfig()
eval_config.batch_size = 10
eval_config.num_steps = 1

with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                               config.init_scale)
    with tf.name_scope("Train"):
        train_input = Input(config=config, data=corpus[20000:], name="TrainInput")
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            m = Seq2SeqModel(is_training=True, config=config, input_=train_input)
        tf.scalar_summary("Training Loss", m.cost)
        tf.scalar_summary("Learning Rate", m.lr)
        
    with tf.name_scope("Valid"):
        valid_input = Input(config=config, data=corpus[10000:15000], name="ValidInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mvalid = Seq2SeqModel(is_training=False, config=config, input_=valid_input)
        tf.scalar_summary("Validation Loss", mvalid.cost)
        
    with tf.name_scope("Test"):
        valid_input = Input(config=config, data=corpus[15000:20000], name="TestInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mtest = Seq2SeqModel(is_training=False, config=eval_config, input_=valid_input)
        tf.scalar_summary("Test Loss", mtest.cost)

    sv = tf.train.Supervisor(logdir="{}/tflog".format(DATA_PATH))
    with sv.managed_session() as session:
        for i in range(config.max_max_epoch):
            lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
            m.assign_lr(session, config.learning_rate * lr_decay)

            print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
            train_perplexity = run_epoch(session, m, eval_op=m.train_op,
                                         verbose=True)
            valid_perplexity = run_epoch(session, mvalid)
            print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
            
            valid_error = run_epoch(session, mvalid, eval_op=mtest._errors, return_eval=True)
            print("Epoch: %d Valid Error: %.3f" % (i + 1, np.mean(valid_error)))   
        
        # test_probs = run_epoch(session, mtest, eval_op=mtest._predict, return_eval=True)
        
        sv.saver.save(session, "{}/tflog".format(DATA_PATH), global_step=sv.global_step)

In [33]:
all_probs = test_probs

In [34]:
test_probs = all_probs[0]

In [36]:
RE_ident = re.compile('[a-zA-Z]+')
known_identifiers_mask = np.array([1.0 if re.match(RE_ident, w) else 0.0 for w in words])
flag_mask = np.array([1.0 if (w == '<flag>' or w == '<no-flag>') else 0.0 for w in words])

In [37]:
known_mask = np.array([1.0 if (w != '<unk>') else 0.0 for w in words])

In [38]:
def predict_flag(probs):
    new_probs = np.apply_along_axis(
        lambda r: r*flag_mask,
        1,
        probs
    )
    return np.argmax(new_probs, axis=1)

def predict_ident(probs):
    new_probs = np.apply_along_axis(
        lambda r: r*known_identifiers_mask,
        1,
        probs
    )
    return np.argmax(new_probs, axis=1)

def predict_known(probs):
    new_probs = np.apply_along_axis(
        lambda r: r*known_mask,
        1,
        probs
    )
    return np.argmax(new_probs, axis=1)

In [39]:
stringify( np.argmax(test_probs, axis=1) )

u'<unk> : <unk> RegExp ( [String] + <unk> + [String] , , <unk> : <unk> RegExp ( [String] + <unk> <unk> ( ( apply , <unk> ( <unk> , <unk> ) { <unk> . $set ( <unk> , <unk> ) ; , , <unk> . $on ( [Keyword|function] . <unk> ( ) ) { [Keyword|return] <unk> . globals ( <unk> , , <unk> . each ( <unk> = [Keyword|function] . getTrusted ( [Keyword|function] ( <unk> ) <unk> ) { [Keyword|return] . parent ( [String] + <unk> + [String] + , join > <unk> . length ( { ) , <unk> <unk> <unk> || $attr ( [String] ) ) , <unk> = <unk> . slice ( <unk> ) type ) ? <unk> [Numeric] : [Numeric] . <unk> ? <unk> ( shiftKey === <unk> ? <unk> !== <unk> ) <unk> ) || . [Numeric] ] || <unk> && <unk> : <unk> === [Keyword|function] <unk> = { <unk> ( , <unk> = prototype <unk> ] expando ( ) ] ] <unk> = <unk> [ <unk> [ replace ( <unk> , substring , <unk> top : <unk> === <unk> . [Numeric] === <unk> ) [String] === <unk> ) [Keyword|return] [Keyword|var] ( <unk> === [Keyword|typeof] . parent ( [Numeric] ) <unk> ) , [Keyword|functio

In [43]:
predict_flag(test_probs)

array([200089, 200089, 200089, 200089, 200089, 200089, 200089, 200089,
       200089, 200089, 200089, 200089, 200088, 200088, 200089, 200089,
       200089, 200089, 200089, 200089, 200089, 200089, 200089, 200088,
       200089, 200089, 200088, 200089, 200089, 200089, 200089, 200089,
       200089, 200089, 200088, 200088, 200089, 200089, 200089, 200089,
       200089, 200089, 200089, 200089, 200089, 200088, 200089, 200089,
       200089, 200089, 200089, 200089, 200089, 200089, 200089, 200089,
       200089, 200089, 200088, 200089, 200089, 200089, 200089, 200089,
       200088, 200089, 200089, 200088, 200089, 200089, 200088, 200089,
       200089, 200088, 200089, 200089, 200089, 200089, 200089, 200089,
       200089, 200088, 200088, 200089, 200089, 200089, 200089, 200089,
       200088, 200089, 200088, 200089, 200089, 200089, 200089, 200088,
       200089, 200089, 200089, 200089, 200089, 200089, 200089, 200088,
       200088, 200088, 200089, 200089, 200089, 200089, 200089, 200089,
      